<a href="https://colab.research.google.com/github/Dillstan/ml_group_project/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##

from scipy.io import loadmat
data = loadmat('imdb.mat')

In [ ]:
import numpy as np

imdb = data['imdb']


# Extract
file_paths_raw = np.squeeze(imdb['full_path'])
celeb_ids_raw  = np.squeeze(imdb['celeb_id'])

print (file_paths_raw)

[[array(['01/nm0000001_rm124825600_1899-5-10_1968.jpg'], dtype='<U43')
  array(['01/nm0000001_rm3343756032_1899-5-10_1970.jpg'], dtype='<U44')
  array(['01/nm0000001_rm577153792_1899-5-10_1968.jpg'], dtype='<U43')
  ...
  array(['08/nm3994408_rm926592512_1989-12-29_2011.jpg'], dtype='<U44')
  array(['08/nm3994408_rm943369728_1989-12-29_2011.jpg'], dtype='<U44')
  array(['08/nm3994408_rm976924160_1989-12-29_2011.jpg'], dtype='<U44')]]


In [ ]:
import cv2
import tensorflow as tf

IMG_SIZE = (160, 160)

def load_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    return img / 255.0

In [ ]:
paths = [str(p[0]) for p in file_paths_raw.item()[0]]
labels = celeb_ids_raw.item()[0].astype(int)

dataset = tf.data.Dataset.from_tensor_slices((paths, labels))

def load_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img /= 255.0
    return img, label

dataset = dataset.map(load_and_preprocess).shuffle(10000).batch(32)

TypeError: iteration over a 0-d array

In [ ]:
from tensorflow.keras import layers, models

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(*IMG_SIZE,3),
    include_top=False,
    weights='imagenet'
)

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')  # num_classes = unique celeb IDs
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(dataset, epochs=10)

In [ ]:
pred = model.predict(img_tensor[np.newaxis,...])
pred_id = np.argmax(pred)
print("Predicted celeb_id:", pred_id)

# Map id to name if needed:
celeb_names = np.squeeze(imdb['celeb_names'])
print("Predicted Name:", celeb_names[pred_id][0])
